In [1]:
import requests
import os
import re
import time
from datetime import datetime
import unidecode
import uuid
from urllib.parse import urlencode
from urllib.parse import urljoin

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
from pandas.errors import EmptyDataError
import numpy as np

# wiijob

### Definition des fonctions

In [2]:
def clean_location(location_string):
    """
    Fonction pour nettoyer une chaîne de localisation en supprimant les espaces inutiles.
    
    Args:
    - location_string (str): Chaîne de localisation à nettoyer.
    
    Returns:
    - str: Chaîne de localisation nettoyée.
    """
    # Supprimer les espaces inutiles et découper par virgule
    cleaned_parts = [part.strip() for part in location_string.split(',')]

    # Réassembler la chaîne avec une virgule et un espace
    cleaned_location = ', '.join(cleaned_parts)

    return cleaned_location

In [82]:
def get_soup(url, params=None):
    """
    Fonction pour récupérer et parser le contenu HTML d'une URL donnée avec BeautifulSoup.
    
    Args:
    - url (str): L'URL de la page à scraper.
    
    Returns:
    - BeautifulSoup object: L'objet BeautifulSoup contenant le contenu HTML parsé.
      None si la requête échoue.
    """
    
    full_link = url
    
    if params:
        full_params = urlencode(params)
        full_link = urljoin(url, '?' + full_params)
    
    print(full_link)
        
    # Faire une requête HTTP GET
    response = requests.get(full_link)

    # Vérifier si la requête s'est bien passée
    if response.status_code == 200:
        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    else:
        print(f"Erreur {response.status_code} lors de la requête HTTP.")
        return None

In [61]:
def get_jobs(soup):
    """
    Fonction pour récupérer les offres d'emploi à partir d'une URL donnée.
    
    Args:
    - url (str): L'URL de la page d'offres d'emploi.
    
    Returns:
    - list: Une liste de dictionnaires contenant les détails de chaque offre.
    """
    job_list = []

    # Vérifier si la requête s'est bien passée
    if soup:
        # Trouver toutes les offres d'emploi
        job_content = soup.find_all('div', class_='jobs-wrapper items-wrapper')
        if job_content:
            job_items = soup.find_all('div', class_='item-job col-sm-12 col-md-12 col-xs-12 lg-clearfix md-clearfix')

            for job_item in job_items:
                job_details = {}
    
                # Titre de l'offre
                job_title_tag = job_item.find('h2', class_='job-title')
                job_details['title'] = job_title_tag.text.strip()
    
                # Lien de l'offre
                job_details['link'] = job_title_tag.a['href'] if job_title_tag.a else ''
    
                # Texte du lien
                job_details['poste'] = job_title_tag.a.text.strip() if job_title_tag.a else ''
    
                # Nom de l'employeur
                employer_name = job_item.find('h2', class_='employeur_name_joblist').text.strip()
                job_details['employer'] = employer_name
    
                # Catégorie du poste
                job_details['category'] = job_item.find('div', class_='job-category with-icon').a.text.strip()
    
                # Lieu de l'offre
                location_string = job_item.find('div', class_='job-location').text.strip()
                job_details['location'] = clean_location(location_string)
    
                # Type de contrat
                job_details['contract_type'] = job_item.find('div', class_='job-type with-title').a.text.strip()
    
                # Date limite
                job_details['deadline'] = job_item.find('div', class_='job-deadline with-icon').text.strip()
    
                # Ajouter les détails de l'offre à la liste
                job_list.append(job_details)
    
    return job_list

In [100]:
def get_pagination_links(soup):
    """
    Fonction pour récupérer les liens de pagination à partir d'un objet BeautifulSoup.
    
    Args:
    - soup (BeautifulSoup): L'objet BeautifulSoup contenant le contenu HTML parsé.
    
    Returns:
    - list: Une liste contenant tous les liens de pagination trouvés.
    """

    pagination_links = set()

    # Vérifier si l'objet BeautifulSoup est valide
    if soup:
        # Trouver le conteneur de pagination
        pagination_wrapper = soup.find('div', class_='jobs-pagination-wrapper main-pagination-wrapper')

        if pagination_wrapper:
            # Trouver tous les liens de pagination dans le conteneur
            pagination_items = pagination_wrapper.find_all('a', class_='page-numbers')

            for item in pagination_items:
                link = item.get('href')
                if link:
                    pagination_links.add(link)

    return list(pagination_links)

In [88]:
query_params = {
    'jobs_ppp': 5,
    'filter-category': '137',
    'filter-location': '368'
    #'filter-career_level': 'Stagiaire',
    #'filter-cfield-job-mode': 'Sur site',
    #'filter-experience': 'Débutant'
}

In [89]:
url = 'https://wiijob.com/offres-emploi/'
soup = get_soup(url, query_params)

https://wiijob.com/offres-emploi/?jobs_ppp=5&filter-category=137&filter-location=368


In [72]:
if soup:
    # Vous pouvez maintenant travailler avec soup pour extraire les données souhaitées
    job_titles = soup.find_all('h2', class_='title')
    for title in job_titles:
        print(title.text.strip())
else:
    print("Impossible de récupérer le contenu HTML.")

BOOSTEZ VOTRE CARRIÈRE


In [90]:
jobs = get_jobs(soup)
len(jobs)

5

In [91]:
jobs[0]

{'title': 'Conseiller Sylviculture et Agroforesterie',
 'link': 'https://wiijob.com/offre-emploi/conseiller-sylviculture-et-agroforesterie/',
 'poste': 'Conseiller Sylviculture et Agroforesterie',
 'employer': 'GIZ – Côte d’Ivoire',
 'category': 'Conseiller du travail',
 'location': "Côte d'Ivoire",
 'contract_type': 'CDD',
 'deadline': '25/06/2024'}

In [101]:
pagination_links = get_pagination_links(soup)
for link in pagination_links:
    print(link)

https://wiijob.com/offres-emploi/page/2/?jobs_ppp=5&filter-category=137&filter-location=368
https://wiijob.com/offres-emploi/page/3/?jobs_ppp=5&filter-category=137&filter-location=368
https://wiijob.com/offres-emploi/page/4/?jobs_ppp=5&filter-category=137&filter-location=368
